In [ ]:
% run utils.ipynb

In [ ]:
import torch.utils.data as data_utils
from sklearn.metrics import accuracy_score, log_loss
from sklearn.preprocessing import StandardScaler
from tensorboardX import SummaryWriter
import skimage.transform as transform
import networkx as nx
from scipy import spatial
import pytorch_fft.fft as fft

In [ ]:
size = 75

In [ ]:
df = pd.read_json('train.json')
df.head()

In [ ]:
s = StandardScaler()
for r in df.band_1:
    s.partial_fit(np.reshape(r, (-1, 1)))

In [ ]:
b = transform.rescale(np.reshape(df.band_1[100], (75, 75)), 0.5, mode='constant')
plt.imshow(b);

In [ ]:
def grid(m):
    idx = np.arange(m)
    return np.reshape(np.meshgrid(idx, idx), (2, -1)).T

In [ ]:
def distance(z, k=4, metric='euclidean'):
    d = spatial.distance.pdist(z, metric)
    d = spatial.distance.squareform(d)
    
    width = d.mean() # preserve distribution
    w = np.exp(- np.square(d / width))
    np.fill_diagonal(w, 0)
    
    cut = w < np.sort(w, axis=1)[:, -k]
    w[cut & np.transpose(cut)] = 0
    return w

In [ ]:
def laplacian(weights):
    degrees = np.sum(weights, axis=1)
    laplacian = np.diag(degrees) - weights
    inv_D_sqrt = np.linalg.inv(np.diag(degrees)) ** (1/2)
    laplacian = inv_D_sqrt @ laplacian @ inv_D_sqrt
    return laplacian

In [ ]:
plt.spy(distance(grid(3), k=3, metric='cityblock'));

In [ ]:
rd = spatial.distance.squareform(spatial.distance.pdist(b.reshape(-1, 1), metric='euclidean'))
plt.matshow(rd);

In [ ]:
ba = b.copy()
rd[np.where(distance(grid(38), k=2) == 0)] = 0

plt.subplot(121)
plt.imshow(rd[:100, :100])

plt.subplot(122)
plt.spy(rd);

In [ ]:
lap = laplacian(rd)

plt.spy(lap[:100, :100])

In [ ]:
def fourier(laplacian):
    return sp.linalg.svd(laplacian)[0]

In [ ]:
plt.matshow(fourier(lap));

In [ ]:
G = nx.from_numpy_array(rd)

nx.draw(nx.subgraph(G, list(G.nodes())[:38 * 2]))

In [ ]:
import coarsening

In [ ]:
G = sp.sparse.csr.csr_matrix(distance(grid(3), 2, metric='cityblock'))

In [ ]:
plt.spy(G.todense())

In [ ]:
graphs, perm = coarsening.coarsen(G, levels=2, self_connections=False)

In [ ]:
perm

In [ ]:
for g in graphs:
    plt.subplot(121)
    plt.spy(g.todense())
    plt.subplot(122)
    nx.draw(nx.from_numpy_array(g.todense()))
    plt.show()

In [ ]:
gr = grid(38)
gr.shape

In [ ]:
mask = distance(gr, k=3) > 0
plt.spy(mask[:40, :40]);

In [ ]:
graphs, perm = coarsening.coarsen(sp.sparse.csr.csr_matrix(mask), levels=3, self_connections=False)

In [ ]:
import scipy.sparse.csgraph

In [ ]:
graphs[0]

In [ ]:
laps = [sp.sparse.csgraph.laplacian(g, normed=True) for g in graphs[:-1]]

In [ ]:
n = laps[0].shape[0]

In [ ]:
gf = df.copy()
gf = gf.assign(
    band_1=df.band_1
        .apply(lambda x: transform.rescale(np.reshape(x, (75, 75)), 0.5, mode='constant'))
        .apply(lambda x: np.r_[x[tuple(gr.reshape(2, -1))], np.zeros(n - 38 * 38)][perm])
)

In [ ]:
def tensorify(col):
    data = s.transform(np.stack(col.values))
    return torch.from_numpy(data.reshape(-1, n)).float()

In [ ]:
split_train_test = np.random.rand(len(gf)) < 0.9
train = gf[split_train_test]
test = gf[~split_train_test]

In [ ]:
train_x = tensorify(train.band_1)
test_x = tensorify(test.band_1)
train_y = torch.from_numpy(train.is_iceberg.values.reshape(-1, 1)).float()
test_y = torch.from_numpy(test.is_iceberg.values.reshape(-1, 1)).float()

In [ ]:
cuda = False#torch.cuda.is_available()
num_epochs = 1
batch_size = 2
learning_rate = 0.00005
cuda

In [ ]:
train_loader = data_utils.DataLoader(data_utils.TensorDataset(train_x, train_y), batch_size=batch_size, shuffle=True)
test_loader = data_utils.DataLoader(data_utils.TensorDataset(test_x, test_y), batch_size=batch_size, shuffle=True)

In [ ]:
f = fourier(laps[0].todense())

In [ ]:
test = Variable(next(train_loader.__iter__())[0])
print(test)

In [ ]:
x = torch.FloatTensor(5,3,4,1)
y = torch.FloatTensor(  3,1,1)
torch.matmul(x, y).size()

In [ ]:
x = torch.from_numpy(np.array([[[1], [2]], [[3], [1]]])).float().transpose(0, 2)
y = torch.from_numpy(np.array([[[1, 2, 3]], [[3, 2, 1]]])).float().transpose(0, 2)
print(y)
print(x)
y @ x

In [ ]:
class GraphConv(nn.Module):
    def __init__(self, fourier, n_filter):
        super(GraphConv, self).__init__()
        
        self.n = len(fourier)
        self.u = Variable(torch.from_numpy(fourier).float(), requires_grad=False)        
        self.n_filter = n_filter
        
        self.weight = nn.Parameter(torch.Tensor(self.n, self.n_filter))
        self.bias = nn.Parameter(torch.Tensor(self.n))
        
        stdv = 1. / np.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        self.bias.data.uniform_(-stdv, stdv)

    def forward(self, x): # samples x n
        # fourier
        out = x @ self.u # samples x n
        
        # filtre
        w = self.weight.unsqueeze(2) # n x f x 1
        out = out.t().unsqueeze(1) # n x 1 x samples
        out = w @ out # n x f x samples
        
        # un-fourier
        out = out.permute(2, 1, 0).contiguous() # samples x f x n
        out = out.view(-1, self.n) # (samples * f) x n
        out = out @ self.u.t() # (samples * f) x n
        out = out.view(-1, self.n_filter, self.n) # samples x f x n
        
        # bias?
        out = out + self.bias # samples x f x n
        return F.relu(out) # samples x f x n

In [ ]:
GraphConv(f, 4)(test)

In [ ]:
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

In [ ]:
net = nn.Sequential(
    GraphConv(f, 2),
    Flatten(),
    nn.Linear(n * 2, 200),
    nn.ReLU(),
    nn.Linear(200, 2),
    nn.Softmax(),
)

In [ ]:
net(test)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
writer = SummaryWriter()

if cuda:
    net.cuda()
    criterion.cuda()
    
epoch_train_loss = []
epoch_test_loss = []

In [ ]:
for e in range(5):

    train_loss = 0
    test_loss = 0

    for batch_id, (x, y) in enumerate(tqdm(train_loader, desc='Training')):
        x = Variable(x)
        y = Variable(y).long().squeeze()
        
        if cuda:
            x = x.cuda()
            y = y.cuda()
        
        optimizer.zero_grad()
        
        outputs = net(x)
        loss = criterion(outputs, y)
        
        train_loss += loss.data.cpu()[0]
        
        loss.backward()
        optimizer.step()
        
    writer.add_scalar('data/scalar1', train_loss, e)
      
    for batch_id, (x, y) in enumerate(tqdm(test_loader, desc='Testing')):
        x = Variable(x)
        y = Variable(y).long().squeeze()
        
        if cuda:
            x = x.cuda()
            y = y.cuda()

        outputs = net(x)
        loss = criterion(outputs, y)

        test_loss += loss.data.cpu()[0]
     
    train_loss /= train.shape[0]
    test_loss /= test.shape[0]
    
    epoch_train_loss.append(train_loss)
    epoch_test_loss.append(test_loss)
    print('Training loss: {:.4f}'.format(train_loss))
    print('Testing  loss: {:.4f}'.format(test_loss))
    
writer.add_graph(net, outputs)

In [ ]:
plt.plot(epoch_train_loss, label='train')
plt.plot(epoch_test_loss, label='test')
plt.legend();

In [ ]:
preds = []
targets = []

for batch_id, (x, y) in enumerate(tqdm(train_loader, desc='Training')):
    x = Variable(x)
    y = Variable(y).long().squeeze()

    if cuda:
        x = x.cuda()
        y = y.cuda()
    
    outputs = net(x)
    preds.extend(outputs.data.cpu().numpy().argmax(axis=1))
    targets.extend(y.data.cpu().numpy())

accuracy_score(targets, preds)

In [ ]:
log_loss(targets, preds)

In [ ]:
preds = []
targets = []

for batch_id, (x, y) in enumerate(tqdm(test_loader, desc='Testing')):
    x = Variable(x)
    y = Variable(y).long().squeeze()

    if cuda:
        x = x.cuda()
        y = y.cuda()
    
    outputs = net(x)
    preds.extend(outputs.data.cpu().numpy().argmax(axis=1))
    targets.extend(y.data.cpu().numpy())

accuracy_score(targets, preds)

In [ ]:
log_loss(targets, preds)